<a href="https://colab.research.google.com/github/tranduynhan02/ML_2022-2023/blob/main/Lab_6_20130346_TranDuyNhan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This lab is to deal with classification task using **Random Forests** and **Naïve Bayes** algorithms with/without **Feature Selection**. 

*   **Deadline: 23:59, 25/03/2023**



# Mount Drive

In [37]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/Study/MachineLearning/lab6'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Study/MachineLearning/lab6


# Import libraries

In [40]:
# code
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn import svm
from sklearn import tree
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from prettytable import PrettyTable

#Task 1. 
Task 1. Compare the performance of selected classification algorithms including **Random forest**, **NaiveBayes**, and **SVM** with **mnist** dataset based on **accuracy, precision, recall, f1** measures according to **without using selection feature** and **using selection feature**.



In [41]:

# code
ds = datasets.load_iris()
X = ds.data
y = ds.target
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.3)
# Random forest
rdforest = RandomForestClassifier(n_estimators=100)
rdforest.fit(X_train, y_train)
y_pred = rdforest.predict(X_test)
acc_rdf = round(metrics.accuracy_score(y_test, y_pred),4)
prec_rdf = round(metrics.precision_score(y_test, y_pred, average='macro'),4)
recall_rdf = round(metrics.recall_score(y_test, y_pred, average='macro'),4)
f1_rdf = round(metrics.f1_score(y_test, y_pred, average='macro'),4)
# NaiveBayes: GaussianNB 
gauNB = GaussianNB() 
gauNB.fit(X_train, y_train) 
y_pred = gauNB.predict(X_test)
acc_gau = round(metrics.accuracy_score(y_test, y_pred),4)
prec_gau = round(metrics.precision_score(y_test, y_pred, average='macro'),4)
recall_gau = round(metrics.recall_score(y_test, y_pred, average='macro'),4)
f1_gau = round(metrics.f1_score(y_test, y_pred, average='macro'),4)
# SVM
s = svm.SVC()
s.fit(X_train, y_train)
y_pred = s.predict(X_test)
acc_s = round(metrics.accuracy_score(y_test, y_pred),4)
precision_s = round((metrics.precision_score(y_test, y_pred, average = 'macro')),4)
f1_s = round((metrics.f1_score(y_test, y_pred, average = 'macro')),4)
recall_s = round((metrics.recall_score(y_test, y_pred, average = 'macro')),4)

t = PrettyTable(['','acc','precision','recall','f1'])
t.add_row(['Random forest',acc_rdf,prec_rdf,recall_rdf,f1_rdf])
t.add_row(['NaiveBayes: GaussianNB ',acc_gau,prec_gau,recall_gau,f1_gau])
t.add_row(['SVM',acc_s,precision_s,recall_s,f1_s])
print(t)

+-------------------------+--------+-----------+--------+--------+
|                         |  acc   | precision | recall |   f1   |
+-------------------------+--------+-----------+--------+--------+
|      Random forest      | 0.9111 |   0.9196  | 0.909  | 0.9117 |
| NaiveBayes: GaussianNB  | 0.9111 |   0.9196  | 0.909  | 0.9117 |
|           SVM           | 0.9556 |   0.9649  | 0.9524 | 0.9558 |
+-------------------------+--------+-----------+--------+--------+


#Task 2. 
For given bank dataset (bank.csv) having the following attributes :
1.	**age** (numeric)
2.	**job** : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3.	**marital** : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4.	**education** (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5.	**default**: has credit in default? (categorical: 'no','yes','unknown')
6.	**housing**: has housing loan? (categorical: 'no','yes','unknown')
7.	**loan**: has personal loan? (categorical: 'no','yes','unknown')
8.	**contact**: contact communication type (categorical: 'cellular','telephone')
9.	**month**: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10.	**day_of_week**: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11.	**duration**: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
12.	**campaign**: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13.	**pdays**: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14.	**previous**: number of contacts performed before this campaign and for this client (numeric)
15.	**poutcome**: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
Output variable (desired target):
16.	**y**. has the client subscribed a term deposit? (binary: 'yes','no')



#3.1. Apply StandardScaler() function to columns that contains numerical data ('age', 'balance', 'day', 'campaign', 'pdays', 'previous')

In [42]:
#code
data = pd.read_csv('bank.csv')
data.columns
X = [['age', 'balance', 'day', 'campaign', 'pdays', 'previous']]
data

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,blue-collar,single,primary,no,1,yes,no,cellular,20,apr,257,1,-1,0,unknown,no
11158,39,services,married,secondary,no,733,no,no,unknown,16,jun,83,4,-1,0,unknown,no
11159,32,technician,single,secondary,no,29,no,no,cellular,19,aug,156,2,-1,0,unknown,no
11160,43,technician,married,secondary,no,0,no,yes,cellular,8,may,9,2,172,5,failure,no


In [43]:
scaler = StandardScaler()

data2 = [data['age'],data['balance'],data['day'],data['campaign'],data['pdays'],data['previous']]


df_stand = pd.DataFrame(scaler.fit_transform(data2))
df_stand

,0,1,2,3,4,5,6,7,8,9,...,11152,11153,11154,11155,11156,11157,11158,11159,11160,11161
0,-0.393896,1.630755,-0.379352,-0.400295,0.199207,2.217915,-0.303468,-0.209849,2.205783,-0.436227,...,-0.278062,1.603462,-0.236063,-0.284428,1.188749,1.853480,-0.344853,1.343605,0.075854,2.154340
1,2.235404,1.162799,2.234984,2.235553,2.141478,-0.521862,2.231256,2.223057,-0.470898,2.236038,...,2.228012,1.153367,2.225609,0.468470,-2.084171,-0.617827,2.233183,1.125723,-0.623089,-0.575383
2,-0.456060,-0.538858,-0.455931,-0.454732,-0.532879,-0.195698,-0.467210,-0.480729,-0.099136,-0.447841,...,-0.411813,-0.309440,-0.466028,-0.518664,0.355081,0.849512,-0.430292,0.399450,-0.493053,0.147191
3,-0.460665,-0.709024,-0.464440,-0.459087,-0.577701,-0.391397,-0.483584,-0.505810,-0.470898,-0.449425,...,-0.496287,-0.759534,-0.502613,-0.836554,0.293328,-0.617827,-0.474869,-0.835214,-0.590580,-0.495097
4,-0.462967,-0.794107,-0.468694,-0.461264,-0.622523,-0.587095,-0.490134,-0.515843,-0.619602,-0.451537,...,-0.524445,-0.872058,-0.513066,2.007730,0.108068,-0.772283,-0.493442,-1.053096,2.172684,-0.655669
5,-0.461816,-0.751565,-0.466567,-0.460175,-0.607582,-0.521862,-0.486859,-0.510827,-0.545250,-0.451009,...,-0.517406,-0.815796,-0.507840,-0.836554,0.138945,-0.695055,-0.489728,-0.980469,-0.541816,-0.575383


#3.2. Apply Encode Categorical Value (OneHotEncoder) to transfrom categorical data to numerical data ('job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome')

In [44]:
#code
encoder = OneHotEncoder(sparse=False)

cat_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']

data3=[data['job'],data['marital'],data['education'],data['default'],data['housing'],data['loan'],data['contact'],data['month'],data['poutcome']]
# data[cat_cols]
# fit and transform the categorical columns
df_encoded = pd.DataFrame(encoder.fit_transform(data[cat_cols]),columns=encoder.get_feature_names_out(cat_cols))

df_processed = pd.concat([data, df_encoded], axis=1)

# remove the original categorical columns
df_processed.drop(columns=cat_cols, inplace=True)

print(df_processed.columns)

Index(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous',
       'deposit', 'job_admin.', 'job_blue-collar', 'job_entrepreneur',
       'job_housemaid', 'job_management', 'job_retired', 'job_self-employed',
       'job_services', 'job_student', 'job_technician', 'job_unemployed',
       'job_unknown', 'marital_divorced', 'marital_married', 'marital_single',
       'education_primary', 'education_secondary', 'education_tertiary',
       'education_unknown', 'default_no', 'default_yes', 'housing_no',
       'housing_yes', 'loan_no', 'loan_yes', 'contact_cellular',
       'contact_telephone', 'contact_unknown', 'month_apr', 'month_aug',
       'month_dec', 'month_feb', 'month_jan', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
       'poutcome_failure', 'poutcome_other', 'poutcome_success',
       'poutcome_unknown'],
      dtype='object')


/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


#3.3. Apply **Decision tree, Random forest, kNN, NaïveBayes** to preproceed dataset in the previous steps. Then compare the obtained results using **accuracy, precision, recall, f1** measures.

In [45]:
#code

X_train, X_test, y_train, y_test = train_test_split(df_processed.drop('deposit', axis=1), df_processed['deposit'], test_size=0.3, random_state=1)

decision = tree.DecisionTreeClassifier()
decision.fit(X_train, y_train) 
y_predDes = decision.predict(X_test)


clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets
clf.fit(X_train,y_train)
y_predForest = clf.predict(X_test)

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
y_predKnn = neigh.predict(X_test)

model = GaussianNB()
model.fit(X_train, y_train)
y_predNaiveBayes = model.predict(X_test)


t = PrettyTable(["Regression","accuracy","precision","recall","f1"])

t.add_row(["DecisionTree",metrics.accuracy_score(y_test,y_predDes),
           metrics.precision_score(y_test,y_predDes,average='macro'),
           metrics.recall_score(y_test,y_predDes,average='macro'),
           metrics.f1_score(y_test,y_predDes,average='macro')])


t.add_row(["kNN",metrics.accuracy_score(y_test,y_predKnn),
           metrics.precision_score(y_test,y_predKnn,average='macro'),
           metrics.recall_score(y_test,y_predKnn,average='macro'),
           metrics.f1_score(y_test,y_predKnn,average='macro')])

t.add_row(["Forest",metrics.accuracy_score(y_test,y_predForest),
           metrics.precision_score(y_test,y_predForest,average='macro'),
           metrics.recall_score(y_test,y_predForest,average='macro'),
           metrics.f1_score(y_test,y_predForest,average='macro')])
t.add_row(["Naive Bayes",metrics.accuracy_score(y_test,y_predNaiveBayes),
           metrics.precision_score(y_test,y_predNaiveBayes,average='macro'),
           metrics.recall_score(y_test,y_predNaiveBayes,average='macro'),
           metrics.f1_score(y_test,y_predNaiveBayes,average='macro')])


print(t)

+--------------+--------------------+--------------------+--------------------+--------------------+
|  Regression  |      accuracy      |     precision      |       recall       |         f1         |
+--------------+--------------------+--------------------+--------------------+--------------------+
| DecisionTree | 0.7966557181248134 | 0.796109640902027  | 0.7962311917157732 | 0.7961669171380041 |
|     kNN      | 0.7363392057330547 | 0.735841010813109  | 0.7348398435265175 | 0.7351543688776938 |
|    Forest    | 0.8471185428486115 | 0.8479395570083252 | 0.8486449811202014 | 0.8470909352924474 |
| Naive Bayes  | 0.7438041206330248 | 0.7509054749561079 | 0.7386402969277419 | 0.7387679306911467 |
+--------------+--------------------+--------------------+--------------------+--------------------+


#3.4. Using selection feature to above dataset, then compare the classification results with those in Task 2.3. 

In [46]:
#code
importances = clf.feature_importances_
#sort
sorted_idx = np.argsort(importances)[::-1]
#print rank
for idx in sorted_idx:
    print(f"{importances[idx]}")


    #lay tu 0.01
sfm = SelectFromModel(clf, threshold=0.01)
X_train = sfm.fit_transform(X_train, y_train)
X_test = sfm.transform(X_test)




decision = tree.DecisionTreeClassifier()
decision.fit(X_train, y_train) 
y_predDes = decision.predict(X_test)


clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets
clf.fit(X_train,y_train)
y_predForest = clf.predict(X_test)

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
y_predKnn = neigh.predict(X_test)

model = GaussianNB()
model.fit(X_train, y_train)
y_predNaiveBayes = model.predict(X_test)


t = PrettyTable(["Regression","accuracy","precision","recall","f1"])

t.add_row(["DecisionTree",metrics.accuracy_score(y_test,y_predDes),
           metrics.precision_score(y_test,y_predDes,average='macro'),
           metrics.recall_score(y_test,y_predDes,average='macro'),
           metrics.f1_score(y_test,y_predDes,average='macro')])


t.add_row(["kNN",metrics.accuracy_score(y_test,y_predKnn),
           metrics.precision_score(y_test,y_predKnn,average='macro'),
           metrics.recall_score(y_test,y_predKnn,average='macro'),
           metrics.f1_score(y_test,y_predKnn,average='macro')])

t.add_row(["Forest",metrics.accuracy_score(y_test,y_predForest),
           metrics.precision_score(y_test,y_predForest,average='macro'),
           metrics.recall_score(y_test,y_predForest,average='macro'),
           metrics.f1_score(y_test,y_predForest,average='macro')])
t.add_row(["Naive Bayes",metrics.accuracy_score(y_test,y_predNaiveBayes),
           metrics.precision_score(y_test,y_predNaiveBayes,average='macro'),
           metrics.recall_score(y_test,y_predNaiveBayes,average='macro'),
           metrics.f1_score(y_test,y_predNaiveBayes,average='macro')])


print(t)

0.3227661290338863
0.07712467934490862
0.07700298015184287
0.06420499205414025
0.03399547294114231
0.0333752954532953
0.031130812119734032
0.02382638899892778
0.01940461988626842
0.018264766310910143
0.017799395532716616
0.015303737291081851
0.01324664311455709
0.01263325202740512
0.012563286145119392
0.012017620700486441
0.010707110439062847
0.010583452915648455
0.010434926265078717
0.010004942176143936
0.009912072457171335
0.009870733932859466
0.00929491608275004
0.008880671904660393
0.008872788899827408
0.008870043630609399
0.008635279338255513
0.008414496404857202
0.0075334010558157395
0.007347072585278476
0.0073056646163974
0.007282685512309918
0.005940043318522906
0.005844815000203871
0.005760775759301343
0.005443451945815355
0.00531121479480495
0.004966411192753186
0.004506932564378654
0.004306918856427705
0.0042458603619382924
0.004050734690464
0.0037947624254317113
0.0034433228849803433
0.0034241431545228387
0.0028759133104651422
0.002521115908821729
0.0017175463824391454
0.00

#Task 4. 
For a given dataset in the Lab #5 (**credit card dataset**), perform feature selection and thencompare the performance of selected classification algorithms (Decision Tree, kNN, Logistic Regression, SVM, Random Forest and NaiveBayes) based on accuracy, precision, recall, f1 measures.


In [47]:
# code
dataSet = pd.read_csv("creditcard.csv")

X = dataSet.drop('Class', axis=1)
y = dataSet['Class']
#Chuan hoa du lieu
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)



X_train, X_test, y_train, y_test = train_test_split(X_scaled,y, test_size=0.3, random_state=1)

decision = tree.DecisionTreeClassifier()
decision.fit(X_train, y_train) 
y_predDes = decision.predict(X_test)


clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets
clf.fit(X_train,y_train)
y_predForest = clf.predict(X_test)

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
y_predKnn = neigh.predict(X_test)

model = GaussianNB()
model.fit(X_train, y_train)
y_predNaiveBayes = model.predict(X_test)


t = PrettyTable(["Regression","accuracy","precision","recall","f1"])

t.add_row(["DecisionTree",metrics.accuracy_score(y_test,y_predDes),
           metrics.precision_score(y_test,y_predDes,average='macro'),
           metrics.recall_score(y_test,y_predDes,average='macro'),
           metrics.f1_score(y_test,y_predDes,average='macro')])


t.add_row(["kNN",metrics.accuracy_score(y_test,y_predKnn),
           metrics.precision_score(y_test,y_predKnn,average='macro'),
           metrics.recall_score(y_test,y_predKnn,average='macro'),
           metrics.f1_score(y_test,y_predKnn,average='macro')])

t.add_row(["Forest",metrics.accuracy_score(y_test,y_predForest),
           metrics.precision_score(y_test,y_predForest,average='macro'),
           metrics.recall_score(y_test,y_predForest,average='macro'),
           metrics.f1_score(y_test,y_predForest,average='macro')])
t.add_row(["Naive Bayes",metrics.accuracy_score(y_test,y_predNaiveBayes),
           metrics.precision_score(y_test,y_predNaiveBayes,average='macro'),
           metrics.recall_score(y_test,y_predNaiveBayes,average='macro'),
           metrics.f1_score(y_test,y_predNaiveBayes,average='macro')])


print(t)

+--------------+--------------------+--------------------+--------------------+--------------------+
|  Regression  |      accuracy      |     precision      |       recall       |         f1         |
+--------------+--------------------+--------------------+--------------------+--------------------+
| DecisionTree | 0.9990871107053826 | 0.8508528148051704 | 0.8664204998358888 | 0.8584670606493835 |
|     kNN      | 0.9995318516437859 | 0.9717634042978214 | 0.874038907383963  | 0.9172381688347941 |
|    Forest    | 0.9994967405170698 | 0.9515668764429444 | 0.8814170092162777 | 0.9135286200093187 |
| Naive Bayes  | 0.9787343609189753 | 0.5280497292284355 | 0.8858107615281621 | 0.5473030512618944 |
+--------------+--------------------+--------------------+--------------------+--------------------+


#Finally,
Save a copy in your Github. Remember renaming the notebook.